In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT, Permutation
from qiskit_aer import AerSimulator
from qiskit import transpile
from fractions import Fraction
from math import gcd
import numpy as np

def c_amod21(a, power):
    if a != 2:
        raise NotImplementedError("Solo a=2 está implementado por ahora")
    
    U = QuantumCircuit(5)
    for _ in range(power):
        U.swap(0, 1)
        U.swap(1, 2)
        U.swap(2, 3)
        U.swap(3, 4)
        for q in range(5):
            U.x(q)
    
    U = U.to_gate()
    U.name = f"{a}^{power} mod 21"
    c_U = U.control()
    return c_U

def shor_N21(a=2):
    n_count = 8
    qc = QuantumCircuit(n_count + 5, n_count)

    qc.h(range(n_count))
    qc.x(n_count)
    for q in range(n_count):
        qc.append(c_amod21(a, 2**q), [q] + list(range(n_count, n_count + 5)))

    qc.append(QFT(n_count, inverse=True, do_swaps=False), range(n_count))
    qc.measure(range(n_count), range(n_count))
    return qc

def analyze_result(counts, n_count, N, a):
    for result in counts:
        y = int(result, 2)
        phase = y / 2**n_count
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator
        print(f"Resultado binario: {result} → y = {y}")
        print(f"Fase estimada: {phase:.4f} ≈ {frac} → r = {r}")

        if r % 2 == 0:
            guess1 = gcd(pow(a, r//2) - 1, N)
            guess2 = gcd(pow(a, r//2) + 1, N)
            if 1 < guess1 < N:
                print(f"✓ Factor encontrado: {guess1}")
            if 1 < guess2 < N:
                print(f"✓ Factor encontrado: {guess2}")


In [27]:
a = 2
N = 21
n_count = 8
max_attempts = 10

backend = AerSimulator()
found = False

for attempt in range(1, max_attempts + 1):
    qc = shor_N21(a)
    qc_compiled = transpile(qc, backend)
    job = backend.run(qc_compiled, shots=1024)
    result = job.result()
    counts = result.get_counts()

    for result_str in counts:
        y = int(result_str, 2)
        phase = y / 2**n_count
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator

        print(f"Resultado binario: {result_str}")
        print(f"Fase estimada: {frac}")
        print(f"r = {r}")

        if r % 2 != 0:
            print("✗ r impar")
            continue

        guess1 = gcd(pow(a, r // 2) - 1, N)
        guess2 = gcd(pow(a, r // 2) + 1, N)
        if 1 < guess1 < N:
            print(f"✓ Factor encontrado: {guess1}")
            found = True
        if 1 < guess2 < N:
            print(f"✓ Factor encontrado: {guess2}")
            found = True

        if found:
            print(f"\n✓✓ Factores de {N} encontrados: {guess1} × {guess2} = {N}")
            break

    if found:
        break
else:
    print(f"\n No se encontraron factores luego de {max_attempts} intentos.")

Resultado binario: 11111011
Fase estimada: 1
r = 1
✗ r impar
Resultado binario: 11101000
Fase estimada: 19/21
r = 21
✗ r impar
Resultado binario: 00110101
Fase estimada: 4/19
r = 19
✗ r impar
Resultado binario: 01010011
Fase estimada: 6/19
r = 19
✗ r impar
Resultado binario: 00110000
Fase estimada: 3/16
r = 16
✓ Factor encontrado: 3

✓✓ Factores de 21 encontrados: 3 × 1 = 21
